In [1]:
# using Luna
# import PyPlot: plt

# # For parameters see http://lupo-lab.com/Luna.jl/dev/interface.html
a = 150e-6   # Fixed radius
flength = 2.5
gas = :Ar
λ0 = 800e-9
τfwhm = 35e-15
λlims = (100e-9, 4e-6)
trange = 400e-15

# using Luna
# import PyPlot: plt

# a = 125e-6
# flength = 3
# gas = :HeJ
# λ0 = 800e-9
# τfwhm = 10e-15
# λlims = (100e-9, 4e-6)
# trange = 400e-15

4.0e-13

In [2]:
# Scan ranges
# energies = collect(range(50e-6, 150e-6; length=4))
# # energies = collect(50e-6:50e-6:500e-6)
# pressures = collect(0.5:0.1:0.8)

energies = collect(50e-6:50e-6:150e-6)
# energies = collect(range(50e-6, 200e-6; length=16))
pressures = collect(0.6:0.4:1.4)

3-element Vector{Float64}:
 0.6
 1.0
 1.4

In [3]:
energies

3-element Vector{Float64}:
 5.0e-5
 0.0001
 0.00015

In [4]:
pressures

3-element Vector{Float64}:
 0.6
 1.0
 1.4

In [9]:
# Set proc for scans, see http://lupo-lab.com/Luna.jl/dev/modules/Scans.html#Luna.Scans.QueueExec
# Need to pass this to main constructor?
Scans.QueueExec(5)

Luna.Scans.QueueExec(5, "")

In [10]:
# Init scan
scan = Scans.Scan("paralleltest_010223", Scans.QueueExec(5))  # energy=energies)
addvariable!(scan,:pressure, pressures)
addvariable!(scan, :energy, energies)

# Init scan as docs - this seems OK IF Scans.Something is first called, so module loading issue?
# scan = Scan("test", Scans.QueueExec(5))  # energy=energies)
# addvariable!(scan,:pressure, pressures)
# addvariable!(scan, :energy, energies)

2-element Vector{Vector}:
 [0.6, 1.0, 1.4]
 [5.0e-5, 0.0001, 0.00015]

In [11]:
@__DIR__

"/mnt/femtobackSSHFS/DriveSyncShare/projects/fibre_codes_2023"

In [12]:
# @__DIR__ gives the directory of the current file
outputdir = joinpath(@__DIR__, "scanoutput_paralleltest_010223")

runscan(scan) do scanidx, energy, pressure
    prop_capillary(a, flength, gas, pressure; λ0, τfwhm, energy,
                   λlims, trange, scan, scanidx, filepath=outputdir)
end

      From worker 3:	┌ Info: Running scan: paralleltest_010223 (9 points)
      From worker 3:	│ Index: 00001
      From worker 3:	│ Variables:
      From worker 3:	│ 	pressure: 0.6
      From worker 3:	└ 	energy: 5e-05
      From worker 2:	┌ Info: Running scan: paralleltest_010223 (9 points)
      From worker 2:	│ Index: 00002
      From worker 2:	│ Variables:
      From worker 2:	│ 	pressure: 1
      From worker 2:	└ 	energy: 5e-05
      From worker 5:	┌ Info: Running scan: paralleltest_010223 (9 points)
      From worker 5:	│ Index: 00003
      From worker 5:	│ Variables:
      From worker 5:	│ 	pressure: 1.4
      From worker 5:	└ 	energy: 5e-05
      From worker 6:	┌ Info: Running scan: paralleltest_010223 (9 points)
      From worker 6:	│ Index: 00004
      From worker 6:	│ Variables:
      From worker 6:	│ 	pressure: 0.6
      From worker 6:	└ 	energy: 0.0001
      From worker 4:	┌ Info: Running scan: paralleltest_010223 (9 points)
      From worker 4:	│ Index: 00005
      From 

Task (done) @0x00007fbd3477eb30

## Processing

In [13]:
λ, Iλ, zstat, edens, max_peakpower = Processing.scanproc(outputdir) do output
    λ, Iλ = Processing.getIω(output, :λ)
    zstat = Processing.VarLength(output["stats"]["z"])
    edens = Processing.VarLength(output["stats"]["electrondensity"])
    max_peakpower = maximum(output["stats"]["peakpower"])
    Processing.Common(λ), Iλ[:, end], zstat, edens, max_peakpower
end

┌ Warning: scanproc failed for file: /mnt/femtobackSSHFS/DriveSyncShare/projects/fibre_codes_2023/scanoutput_paralleltest_010223/paralleltest_010223_00001.h5:
│ KeyError: key "saved" not found
│ Stacktrace:
│   [1] getindex(parent::HDF5.Group, path::String; pv::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
│     @ HDF5 ~/.julia/packages/HDF5/TcavY/src/highlevel.jl:63
│   [2] getindex(parent::HDF5.Group, path::String)
│     @ HDF5 ~/.julia/packages/HDF5/TcavY/src/highlevel.jl:62
│   [3] (::Luna.Output.var"#5#7"{String})(file::HDF5.File)
│     @ Luna.Output ~/.julia/packages/Luna/cyPdO/src/Output.jl:174
│   [4] (::HDF5.var"#17#18"{HDF5.HDF5Context, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}, Luna.Output.var"#5#7"{String}, HDF5.File})()
│     @ HDF5 ~/.julia/packages/HDF5/TcavY/src/file.jl:98
│   [5] task_local_storage(body::HDF5.var"#17#18"{HDF5.HDF5Context, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}, Luna.Output.var"#5#7"{String}, H

LoadError: UndefVarError: arrays not defined

In [14]:
size(Iλ)

LoadError: UndefVarError: Iλ not defined

In [15]:
size(λ)

LoadError: UndefVarError: λ not defined

In [16]:
fig, axs = plt.subplots(3, round(Int,length(pressures)/3))
fig.set_size_inches(20, 14)
for (pidx, pressure) in enumerate(pressures)
    ax = axs[pidx]
    global img = ax.pcolormesh(λ*1e9, energies*1e6, 10*Maths.log10_norm(Iλ[:, :, pidx])')
    img.set_clim(-40, 0)
    ax.set_xlabel("Wavelength (nm)")
    ax.set_ylabel("Energy (μJ)")
    ax.set_title("Pressure: $pressure bar")
    ax.set_xlim(100, 1200)
end
plt.colorbar(img, ax=axs, label="Energy density (dB)")

LoadError: UndefVarError: λ not defined